In [1]:
%pip install openpyxl
%pip install pandas 

import gc
from enum import Enum
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
input_columns_file_path = '../data/columns.xlsx'

output_columns_file_path = '../data/interm/3.0_columns.csv'
output_revenue_taxes_file_path = '../data/interm/3.0_all_taxes.csv'

# Extract tax columns that represent commune, syndicate, department, region & state revenues

In [3]:
TAX_CATEGORY_CODES = [
    'TFPNB',  # Taxe Foncière sur les Propriétés Non Bâties - Property Tax on Unbuilt Land
    'TFPB',   # Taxe Foncière sur les Propriétés Bâties - Property Tax on Built Land
    'TAFB',   # Taxe Additionnelle à la Taxe Foncière sur les Propriétés Bâties - Additional Tax on Property Tax for Built Land
    'TASARIF', # Taxe Additionnelle à la Taxe Foncière sur les Propriétés Non Bâties - Additional Tax on Property Tax for Unbuilt Land
    'TAFNB',  # Taxe Additionnelle à la Taxe Foncière sur les Propriétés Non Bâties - Additional Tax on Property Tax for Unbuilt Land
    'CFE',    # Cotisation Foncière des Entreprises - Business Property Tax
    'CVAE',   # Cotisation sur la Valeur Ajoutée des Entreprises - Corporate Value Added Contribution
    'TSE',    # Taxe sur les Salaires - Payroll Tax
    'IFER',   # Imposition Forfaitaire sur les Entreprises de Réseaux - Flat-Rate Tax on Network Companies
    'TASCOM', # Taxe sur les Surfaces Commerciales - Tax on Commercial Surfaces
    'GEMAPI', # Gestion des Milieux Aquatiques et Prévention des Inondations - Management of Aquatic Environments and Flood Prevention
    'FB',     # Foncier Bâti - Property Tax on Built Land
    'TP',     # Taxe Professionnelle - Professional Tax    
    'TF',     # Taxe Foncière - Property Tax
    'TH',     # Taxe d'Habitation - Residence Tax,
    'FNB',    # Foncier Non Bâti - Property Tax on Undeveloped Land
    'BASE NETTE IMPOSEE', # Taxable or relieved net base for secondary residences
    'TEOM'   # Taxe d'Enlèvement des Ordures Ménagères - Household Waste Collection Tax
]

COMMUNE_REVENUE_COLUMNS = [
    # General taxes
    'FB - COMMUNE / MONTANT REEL',
    'FNB - COMMUNE / MONTANT REEL',
    'IFER TOTALE / COMMUNE',
    'TH - COMMUNE / MONTANT REEL COMMUNAL AU PROFIT DE LA COMMUNE',
    'TAFNB - COMMUNE / MONTANT REEL NET',
    'TASCOM au profit de la commune',

    # Taxe Professionnelle - Professional Tax
    'CFE - COMMUNE / PRODUIT REEL NET', # It indicates the actual net revenue from CFE that has been issued to the municipality. It directly reflects the financial inflow from this tax to the commune's budget.
    'CFE - TSE / PRODUIT REEL NET',
    'Part de CVAE au profit de la commune', #  Represents the actual portion of CVAE revenue allocated to the commune. This is a real figure indicating what the commune receives.
    'Part de CVAE dégrevée au profit de la commune', # The amount of CVAE that has been rebated back to the commune, reflecting a real adjustment in the financial records.
    'Part de CVAE exonérée compensée au profit de la commune', # Indicates the amount of CVAE that is exempt but compensated to the commune by other means (possibly state or higher administrative bodies), showing a tangible impact on the commune's finances. 
    
    # Extra/specific taxes 
    'FB - TSE / MONTANT REEL', 
    'FNB - TSE / MONTANT REEL',
    'TH - TSE / MONTANT REEL',
    'IFER EOLIENNES ART 1519 D DU CGI / COMMUNE / MONTANT',
    'IFER HYDROLIENNES ART 1519 D DU CGI / COMMUNE / MONTANT',
    'IFER CENTRALES NUCLEAIRES OU THERMIQUES A FLAMME ART 1519 E DU CGI / COMMUNE / MONTANT',
    'IFER CENTRALES PHOTOVOLTAIQUES ART 1519 F DU CGI / COMMUNE / MONTANT',
    'IFER BARRAGES HYDRAULIQUES ART 1519 F DU CGI / COMMUNE / MONTANT',
    'IFER TRANSFORMATEURS ELECTRIQUES ART 1519 G DU CGI / COMMUNE / MONTANT',
    'IFER STATIONS RADIOELECTRIQUES ART 1519 H DU CGI / COMMUNE / MONTANT',
    'IFER INSTALLATIONS DE GAZ NATUREL ART 1519HA DU CGI / COMMUNE / MONTANT',
    'IFER GEOTHERMIE (Art 1519 HB) DU CGI / COMMUNE / MONTANT',
    'TH - MONTANT REEL DE LA MAJORATION DES RESIDENCES SECONDAIRES',
]

SYNDICAT_REVENUE_COLUMNS = [
    'FNB - SYNDICATS ET ORG.ASSIMILES /  MONTANT REEL',
    'FB - SYNDICATS ET ORG. ASSIMILES /  MONTANT REEL',
    'FB - SYNDICATS ET ORG. ASSIMILES /  MONTANT LISSAGE',
    'FB - TAXE INCITATIVE ENLEVEMENT DES ORDURES MENAGERES / MONTANT REEL / SYNDICAT',
    'TF - MONTANT DES DO JEUNES AGRICULTEURS - Syndicats (COLLECTIVITES)',
    'TH - SYNDICATS ET ORG. ASSIMILES /  MONTANT REEL',
    'TH - SYNDICATS ET ORG. ASSIMILES /  MONTANT LISSAGE',
    'CFE - LISSAGE / SYNDICATS / MONTANT',
    'TH - SYNDICATS ET ORG. ASSIMILES /  PRODUIT REEL DES LOCAUX VACANTS',
    'CFE - SYNDICATS ET ORG. ASSIMILES /  PRODUIT REEL NET',
]

INTERCOMMUNE_REVENUE_COLUMNS = [
    'CFE - INTERCOMMUNALITE / PRODUIT REEL NET / FISCALITE ADDITIONNELLE OU FP DE ZONE (HORS ZONE)',
    'CFE - INTERCOMMUNALITE / PRODUIT REEL NET / FP UNIQUE OU EN ZAE',
    'CFE - INTERCOMMUNALITE / PRODUIT REEL NET / FP UNIQUE',
    'CFE - INTERCOMMUNALITE / PRODUIT REEL NET / FPZ EN ZAE',
    'CFE - INTERCOMMUNALITE / PRODUIT REEL NET / FPE EN ZONE EOLIENNE',
    'CFE - GEMAPI / PRODUIT REEL NET INTERCOMMUNAL / FISCALITE ADDITIONNELLE',
    'CFE - GEMAPI / PRODUIT REEL NET INTERCOMMUNAL / FP UNIQUE OU EN ZAE',
    'CFE - GEMAPI / PRODUIT REEL NET INTERCOMMUNAL / FP UNIQUE',
    'CFE - GEMAPI / PRODUIT REEL NET INTERCOMMUNAL / FPZ EN ZAE',
    'CFE - GEMAPI / PRODUIT REEL NET INTERCOMMUNAL / FPE EN ZONE EOLIENNE',

    'FB - GFP / MONTANT REEL',
    'FB - GEMAPI / MONTANT REEL INTERCOMMUNALITE',
    'FNB - GEMAPI / MONTANT REEL INTERCOMMUNALITE',
    'FNB - GFP / MONTANT REEL',

    'IFER TOTALE / INTERCOMMUNALITE',
    'IFER EOLIENNES ART 1519 D DU CGI / EPCI A FISCALITE ADDITIONNELLE / MONTANT',
    'IFER EOLIENNES ART 1519 D DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',
    'IFER EOLIENNES ART 1519 D DU CGI / EPCI A REGIME EOLIEN / MONTANT',
    'IFER HYDROLIENNES ART 1519 D DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',
    'IFER HYDROLIENNES ART 1519 D DU CGI / EPCI A REGIME EOLIEN / MONTANT',
    'IFER CENTRALES NUCLEAIRES OU THERMIQUES A FLAMME ART 1519 E DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',
    'IFER CENTRALES PHOTOVOLTAIQUES ART 1519 F DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',
    'IFER BARRAGES HYDRAULIQUES ART 1519 F DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',
    'IFER TRANSFORMATEURS ELECTRIQUES ART 1519 G DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',
    'IFER STATIONS RADIOELECTRIQUES ART 1519 H DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',
    'IFER INSTALLATIONS DE GAZ NATUREL ART 1519HA DU CGI / EPCI A FISCALITE ADDITIONNELLE / MONTANT',
    'IFER INSTALLATIONS DE GAZ NATUREL ART 1519HA DU CGI / EPCI A FISCALITE UNIQUE OU ZAE / MONTANT',

    'Part de CVAE au profit du GFP',

    'TH - INTERCOMMUNALITE / MONTANT REEL DONT THP/E AU PROFIT DE L ETAT',
    'TH - INTERCOMMUNALITE / MONTANT REEL INTERCOMMUNALITE DE THP/E AU PROFIT DE L ETAT',
    'TH - INTERCOMMUNALITE / MONTANT REEL INTERCOMMUNALITE AU PROFIT DU GROUPEMENT',
    'TH - MONTANT REEL INTERCOMMUNAL TAXE GEMAPI',
    'TH - MONTANT NET INTERCOMMUNAL DES LOGEMENTS VACANTS TAXE GEMAPI',
    
    'TASCOM au profit du GFP',
]

DEPARTMENT_REVENUE_COLUMNS = [
    'FNB - CHAMBRE D\'AGRICULTURE / MONTANT REEL',

    'IFER EOLIENNES ART 1519 D DU CGI / DEPARTEMENT / MONTANT',
    'IFER HYDROLIENNES ART 1519 D DU CGI / DEPARTEMENT / MONTANT',
    'IFER CENTRALES NUCLEAIRES OU THERMIQUES A FLAMME ART 1519 E DU CGI / DEPARTEMENT / MONTANT',
    'IFER CENTRALES PHOTOVOLTAIQUES ART 1519 F DU CGI / DEPARTEMENT / MONTANT',
    'IFER BARRAGES HYDRAULIQUES ART 1519 F DU CGI / DEPARTEMENT / MONTANT',
    'IFER INSTALLATIONS DE GAZ NATUREL ART 1519HA DU CGI / DEPARTEMENT / MONTANT',
    'IFER STATIONS RADIOELECTRIQUES ART 1519 H DU CGI / DEPARTEMENT / MONTANT',
    'IFER TOTALE / DEPARTEMENT',

    'Part de CVAE au profit du département'
]

REGION_REVENUE_COLUMNS = [
    'CFE - TASA / PRODUIT REEL NET', # TSE taxable base for CFE on the territory of the municipality. A special annual additional tax is levied for the benefit of the Ile-de-France region. Proceeds are allocated to the investment section of the region's budget. The tax base is determined under the same conditions as for the main tax, to which the additional tax is added.
    'CFE - CHAMBRE DE COMMERCE ET INDUSTRIE /  PRODUIT REEL NET',
    'CFE - CHAMBRE DES METIERS /  DROIT ADDITIONNEL / PRODUIT NET',

    'Part de CVAE au profit de la région',

    'IFER TOTALE / REGION',
    'IFER GEOTHERMIE (Art 1519 HB) DU CGI / REGION / MONTANT',
    'IFER MATERIEL ROULANT FERROVIAIRE ART 1599 QUATER A DU CGI / REGION / MONTANT DECLARE PAR LES SOCIETES DE TRANSPORT FERROVIAIRE NATIONAL',
    'IFER REPARTITEURS PRINCIPAUX ART 1599 QUATER B DU CGI / REGION / MONTANT'
]

MGP_REVENUE_COLUMNS = [
    'FB - TSE AUTRES / MONTANT REEL NET',
    'FNB - TSE GRAND PARIS OU EPFL GUADELOUPE OU EPFL MARTINIQUE / MONTANT REEL',
    'IFER MATERIEL ROULANT RATP ART 1599 QUATER A BIS DU CGI / GRAND PARIS / MONTANT',
    'TH - TSE GRAND PARIS OU EPFL GUADELOUPE OU EPFL MARTINIQUE / MONTANT REEL',
]

STATE_REVENUE_COLUMNS = [
    'TH - COMMUNE / MONTANT REEL DONT THP/E AU PROFIT DE L ETAT',
    'TH - COMMUNE / MONTANT REEL COMMUNAL DE THP/E AU PROFIT DE L ETAT'
]

OTHER_REVENUE_COLUMNS = [
    'FNB - CAAA / DROIT FIXE - MONTANT REEL', # The tax for the benefit of the Caisse d'Assurance Accidents Agricoles (CAAA) is levied only in the Moselle
    'FNB - CAAA / DROIT PROPORTIONNEL -  MONTANT REEL'
]

class RevenueCollector(Enum):
    COMMUNE = "COMMUNE"
    SYNDICAT = "SYNDICAT"
    INTERCOMMUNE = "INTERCOMMUNE"
    DEPARTMENT = "DEPARTMENT"
    REGION = "REGION"
    MGP = "MGP"
    STATE = "STATE"
    OTHER = "OTHER"
    UNAVAILABLE="UNAVAILABLE"

all_revenue_columns = (
    COMMUNE_REVENUE_COLUMNS +
    SYNDICAT_REVENUE_COLUMNS +
    INTERCOMMUNE_REVENUE_COLUMNS +
    DEPARTMENT_REVENUE_COLUMNS +
    REGION_REVENUE_COLUMNS +
    MGP_REVENUE_COLUMNS +
    STATE_REVENUE_COLUMNS +
    OTHER_REVENUE_COLUMNS
)

print("The number of the commune revenue columns is =", len(COMMUNE_REVENUE_COLUMNS))
print("The number of the syndicat revenue columns is =", len(SYNDICAT_REVENUE_COLUMNS))
print("The number of the intercommune revenue columns is =", len(INTERCOMMUNE_REVENUE_COLUMNS))
print("The number of the department revenue columns is =", len(DEPARTMENT_REVENUE_COLUMNS))
print("The number of the region revenue columns is =", len(REGION_REVENUE_COLUMNS))
print("The number of the MGP revenue columns is =", len(MGP_REVENUE_COLUMNS))
print("The number of the state revenue columns is =", len(STATE_REVENUE_COLUMNS))
print("The number of the other revenue columns is =", len(OTHER_REVENUE_COLUMNS))
print("Total number of tax revenue columns is =", len(all_revenue_columns))

The number of the commune revenue columns is = 24
The number of the syndicat revenue columns is = 10
The number of the intercommune revenue columns is = 34
The number of the department revenue columns is = 10
The number of the region revenue columns is = 8
The number of the MGP revenue columns is = 4
The number of the state revenue columns is = 2
The number of the other revenue columns is = 2
Total number of tax revenue columns is = 94


In [4]:
df = pd.read_excel(input_columns_file_path)

# Add the new columns
df['DATA_TYPE'] = ''
df['TAX_CATEGORY'] = ''
df['REVENUE_COLLECTOR'] = ''

# Reorder the columns
new_column_order = [
    'TITRE', 'DATA_TYPE', 'TAX_CATEGORY', 'REVENUE_COLLECTOR', 'ENGLISH_DESCRIPTION',
    'NUMERO_DE_VARIABLE', 'NOM_DE_VARIABLE', 'LONGUEUR', 'FORMAT', 'POSITION_DE_DEBUT', 'POSITION_DE_FIN', 'OBSERVATIONS'
]

df = df[new_column_order]


# Determine the revenue collector for each of the selected taxes

In [5]:
def determine_revenue_collector(column):
    revenue_collector = RevenueCollector.UNAVAILABLE

    if column in COMMUNE_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.COMMUNE
    if column in SYNDICAT_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.SYNDICAT
    elif column in INTERCOMMUNE_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.INTERCOMMUNE
    elif column in DEPARTMENT_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.DEPARTMENT
    elif column in REGION_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.REGION
    elif column in MGP_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.MGP
    elif column in STATE_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.STATE
    elif column in OTHER_REVENUE_COLUMNS:
        revenue_collector = RevenueCollector.OTHER
    return revenue_collector.value
    
def extract_additional_info(titre):
    data_type = 'UNKNOWN'
    tax_category = 'UNKNOWN'
    revenue_collector = RevenueCollector.UNAVAILABLE.value

    for category in TAX_CATEGORY_CODES:
        if category in titre:  # Case-insensitive comparison
            data_type = 'TAX'
            tax_category = category
            revenue_collector = determine_revenue_collector(titre)
       

    return data_type, tax_category, revenue_collector

for index, row in df.iterrows():
    titre = row['TITRE']
    data_type, tax_category, revenue_collector = extract_additional_info(titre)

    df.at[index, 'DATA_TYPE'] = data_type
    df.at[index, 'TAX_CATEGORY'] = tax_category
    df.at[index, 'REVENUE_COLLECTOR'] = revenue_collector

In [6]:
all_taxes_df = pd.DataFrame(all_revenue_columns, columns=['TITRE'])
all_taxes_df.to_csv(output_revenue_taxes_file_path, index = False)

In [7]:
df.to_csv(output_columns_file_path, index= False)

del df
del all_taxes_df
gc.collect()

138